***Consider the following Link: https://www.imdb.com/search/title/?release_date=2020&sort=num_votes,desc&page=1. The page contains information about movies released in the year 2020 sorted according to number of votes. Scrape the following information for the first 1000 movies: (i) Movie name (ii) IMDB Rating (iii) Description (iv) Votes (v) Director(s) (vi) Stars (vii) Metascore (viii) Movie Genre (ix) Total Duration***

In [ ]:
!pip install bs4
!pip install requests

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
def get_data(pageno):
  global directors, stars
  page=pageno-1
  url=('https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start='+str(1+50*page)+'&ref_=adv_nxt')
  content=requests.get(url).text
  soup=BeautifulSoup(content, 'html.parser')
  #soup=soup.prettify() #for indentation
  table=[]
  for itemcon in soup.find_all('div', class_='lister-item-content'):
    movie_details=[]
    moviename=itemcon.h3.a
    rate=itemcon.find('div', class_='ratings-bar')
    rating=rate.strong
    desc=itemcon.find_all('p', class_='text-muted')
    desp=desc[1]
    vote=itemcon.find('p', class_='sort-num_votes-visible')
    v=vote.find_all('span')
    votes=v[1].text

    ptag=itemcon.find_all('p', class_='')
    if ptag is not None:
      moviecast=ptag[0].text
      x=moviecast.split('|')
      if len(x)==2:
        x1=x[0].replace('\n',''); x1=x1.split(':')
        directors=x1[1]
        x2=x[1].replace('\n',''); x2=x2.split(':')
        stars=x2[1]
      if len(x)==1:
        x1=x[0].replace('\n',''); x1=x1.split(':')
        if x1[0]=="Director" or "Director(s)" or "Directors":
          directors=x1[1]
          stars=None
        if x1[0]=="Star" or "Stars" or "Star(s)":
          stars=x1[1]
          directors=None
    else:
       stars=None
       directors=None
    
    rate=itemcon.find('div', class_='ratings-bar')
    meta=rate.find('div', class_='inline-block ratings-metascore')
    if meta is not None:
      metascore=meta.span.text
    else:
      metascore=None
    t_muted=itemcon.find('p', class_='text-muted')
    genre=t_muted.find('span', class_='genre')
    t_muted=itemcon.find('p', class_='text-muted')
    duration=t_muted.find('span', class_='runtime')
    
    if moviename is not None:
      moviename=itemcon.h3.a.text
      movie_details.append(moviename)
    else:
      movie_details.append(None)
    if rating is not None:
      rating=rate.strong.text
      movie_details.append(rating)
    else:
      movie_details.append(None)
    if desp is not None:
      desp=desc[1].text 
      desp=desp.replace('\n','')
      movie_details.append(desp)
    else:
      movie_details.append(None)
    if votes is not None:
      votes=v[1].text
      movie_details.append(votes)
    else:
      movie_details.append(None)

    if directors is not None:
      movie_details.append(directors)
    else:
      movie_details.append(None)
    if stars is not None:
      movie_details.append(stars)
    else:
      movie_details.append(None)

    if metascore is not None:
      metascore=meta.span.text
      movie_details.append(metascore)
    else:
      movie_details.append(None)
    if genre is not None:
      genre=t_muted.find('span', class_='genre').text
      genre=genre.replace('\n','')
      movie_details.append(genre)
    else:
      movie_details.append(None)
    if duration is not None:
      duration=t_muted.find('span', class_='runtime').text
      movie_details.append(duration)
    else:
      movie_details.append(None)

    table.append(movie_details)
  return table

In [ ]:
!pip install pandas 

In [ ]:
import pandas as pd
pg=[]
for i in range(1,21):
    pg.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
print(flatten(pg))
df = pd.DataFrame(flatten(pg),columns=['Name','Rating','Description','Votes', 'Directors', 'Stars', 'Metascore', 'Genre', 'Duration'])